In [3]:
try:
    ! nest-server start -o -d -p 52425
    RAY = False
except:
    RAY = True
    # TODO: Find out why we can't deamonize the NEST server...
    # For the moment we need to use Ray to deamonize it, or start it on a terminal.
    # ! nest-server start -o -p 52425

    import os
    import ray

    ray.init()  # _temp_dir="/home/docker/tmp/ray", object_store_memory=100000000


    @ray.remote
    def start_deamonize_nest_server():
        os.system('! nest-server start -o -p 52425')

    # @ray.remote
    # def nest_server_status():
    #     os.system('! nest-server status')

    # @ray.remote
    # def nest_server_log():
    #     os.system('! nest-server log')

    start_deamonize_nest_server.remote()
    # nest_server_status.remote()
    # nest_server_log.remote()

NEST Server is now running at http://0.0.0.0:52425.


In [4]:
from nest_client import NESTClient

print('Running client examples using NEST via NEST Server')

# Load NEST Server client
nestsc = NESTClient(host="127.0.0.1", port=52425)

#
# Use NEST Server API
#
print('\n')
print('Execute script code with NEST Server API')
print('-' * 20)

# Reset kernel
nestsc.ResetKernel()
nestsc.GetKernelStatus("biological_time")

# Create nodes
pg = nestsc.Create("poisson_generator", params={"rate": 6500.})
neurons = nestsc.Create("iaf_psc_alpha", 100)
sr = nestsc.Create("spike_recorder")

# Connect nodes
nestsc.Connect(pg, neurons, syn_spec={'weight': 10.})
nestsc.Connect(neurons[::10], sr)

# Simulate
nestsc.Simulate(1000.0)

# Get events
n_events = nestsc.GetStatus(sr, 'n_events')[0]
print('Number of events:', n_events)


#
# Use NEST Server exec
#
print('\n')
print('Execute script code from file')
print('-' * 20)

n_events = nestsc.from_file('/home/docker/packages/nest-client/examples/NESTClient_script.py', 'n_events')['data']
print('Number of events:', n_events)

Running client examples using NEST via NEST Server


Execute script code with NEST Server API
--------------------
Number of events: 126


Execute script code from file
--------------------
Execute script code of /home/docker/packages/nest-client/examples/NESTClient_script.py
Return variables: n_events
--------------------
# -*- coding: utf-8 -*-
#
# NESTClient_script.py
#
# This file is part of NEST.
#
# Copyright (C) 2004 The NEST Initiative
#
# NEST is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 2 of the License, or
# (at your option) any later version.
#
# NEST is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# al

In [5]:
from tvb_multiscale.tvb_nest.nest_models.server_client.nest_server_client import NESTServerClient 

# Load NEST Server client
nest = NESTServerClient(host='localhost', port=52425)

nest.GetKernelStatus("biological_time")

1000.0

In [6]:
! curl localhost:52425

{"mpi":false,"nest":"3.5"}


In [7]:
import requests

requests.get('http://localhost:52425').json()
requests.get('http://127.0.0.1:52425/api/GetKernelStatus', json=["biological_time"]).json()

1000.0

In [8]:
! nest-server stop

NEST Server running at http://0.0.0.0:52425 has stopped.


In [9]:
if RAY: 
    ray.shutdown()